Here, we want to get the heuristic labels for the images that humans labeled.

In [1]:
import pandas as pd
import urllib.parse
import os
from tqdm import tqdm
import help_functions as hf
import PIL
from PIL import Image
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../')
from config import FILES_ANNOTATED_PATH, EVALUATION_PATH
tqdm.pandas()

Using TensorFlow backend.


In [2]:
# Input files --> update this to the latest data file generated with Francesco's taxonomy in get_labels.ipynb
# LABELS_PATH = 'data/commonswiki-221218-files-hierarchical-labels.json.bz2'
LABELS_PATH = FILES_ANNOTATED_PATH

# Output files --> choose 
CLEANED_LABELS_PATH = 'data/commonswiki-230412-clean-labels.json.bz2'
SPLIT_DATA_PATH = 'data/split_data_230412'

labels = pd.read_parquet(LABELS_PATH).rename(columns={'labels_pred': 'labels'})
print(labels.shape)
labels.head()

(3891446, 5)


,id,title,url,categories,labels
0,80902489,"""A Gathering of Court Women"", Folio from the D...",c/c1/%22A_Gathering_of_Court_Women%22%2C_Folio...,"[Department of Islamic Art, Metropolitan Museu...","[Art, Culture]"
1,53631642,"""Akdeniz"" - Aliağa, 2016.jpg",f/f8/%22Akdeniz%22_-_Alia%C4%9Fa%2C_2016.jpg,"[Akdeniz (ship, 1955), Aliağa ship-breaking yard]",[]
2,74785910,"""Carnevale di Torino"". Felice Cerruti - f.lli ...",1/13/%22Carnevale_di_Torino%22._Felice_Cerruti...,"[Felice Cerruti Bauduc, Victor Emmanuel II of ...","[Culture, Society, People, Places, Art]"
3,14890941,"""Escena de playa con figura"".jpg",c/c2/%22Escena_de_playa_con_figura%22.jpg,"[Pedro Lira, People with dogs in art]","[Art, People, Culture, Society]"
4,17828296,"""Florero con plátanos, limones y libros"", Juan...",d/da/%22Florero_con_pl%C3%A1tanos%2C_limones_y...,"[Works by Juan de Echevarría, Still-life paint...","[Art, Culture]"


In [3]:
EVALUATION_PATH = '../../data/evaluation/'
held_out_test = pd.read_parquet(EVALUATION_PATH + "annotated_test.parquet")
held_out_val = pd.read_parquet(EVALUATION_PATH + "annotated_validation.parquet")
held_out_set = pd.concat([held_out_test, held_out_val])
print(held_out_set.shape)

(1087, 5)


In [4]:
print(held_out_val.shape)
held_out_val.head()

(327, 5)


,title,id,url,categories,labels
1,"Figure-8---Creme-Angelique,-Illustration.png",30561758,https://upload.wikimedia.org/wikipedia/commons...,[Washington W. Sheffield],"[STEM, Medicine & Health]"
2,0 Dréhance - Château de Walzin (1).jpg,16644463,https://upload.wikimedia.org/wikipedia/commons...,"[Château de Walzin, 2006 in Dinant, Lesse (riv...","[Places, STEM, Landscapes, Nature, Plants]"
3,Douglas geologic map.PNG,48442260,https://upload.wikimedia.org/wikipedia/commons...,"[Douglas Island, The Juneau Gold Belt, Alaska ...","[Places, Maps & Flags]"
4,Cryptochiton stelleri.jpg,9554405,https://upload.wikimedia.org/wikipedia/commons...,[Cryptochiton stelleri],"[STEM, Animals, Nature]"
5,Bydg ul Gdańska - koszary.jpg,23997615,https://upload.wikimedia.org/wikipedia/commons...,"[Gdańska 147-161 Koszary, Zawisza (Bydgoszcz)]","[Architecture, Places]"


In [8]:
human_labeled_data_with_heuristic_labels = labels[labels['title'].isin(held_out_val['title'])]
print(human_labeled_data_with_heuristic_labels.shape)
human_labeled_data_with_heuristic_labels.to_json('human_labeled_data_with_heuristic_labels.json.bz2')
human_labeled_data_with_heuristic_labels.head()

(327, 5)


,id,title,url,categories,labels
15831,90932216,Silverdale War Memorial.jpg,b/b0/Silverdale_War_Memorial.jpg,"[War memorials in New Zealand, Silverdale, Auc...","[History, Monuments & Memorials, Culture, Soci..."
22859,5431865,"Bundesarchiv Bild 183-90569-0001, Berlin, Vors...",6/6c/Bundesarchiv_Bild_183-90569-0001%2C_Berli...,"[1962 in Mitte (district of Berlin), Changing ...","[Culture, Society, Art, STEM, Medicine & Health]"
22912,23997615,Bydg ul Gdańska - koszary.jpg,3/3c/Bydg_ul_Gda%C5%84ska_-_koszary.jpg,"[Gdańska 147-161 Koszary, Zawisza (Bydgoszcz)]","[Architecture, Places]"
62065,44722339,Cedar Point Pink's Hot Dogs (14669390970).jpg,3/37/Cedar_Point_Pink%27s_Hot_Dogs_%2814669390...,"[Food at Cedar Point, July 2014 in Ohio, Cedar...","[Food, Culture, Society]"
71525,19914023,Pelorus River from Pelorus Bridge.jpg,5/51/Pelorus_River_from_Pelorus_Bridge.jpg,"[Pelorus River, Pelorus Bridge]","[Nature, Society, Places, STEM]"


ok, now i have the heuristic predicted labels. Now, i want to predict metrics for them using the model weights we have and predict them graph